# INFERENCE OF MACHINE LEARNING TASKS ON EDGE DEVICES
 - This Notebook is used to train and test Machine Learning models
 - The code is partially borrowed from [LINK](https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/)


## Testing ML Models

# Import Libraries

In [1]:
import os
import sys
import cv2
import time
import numpy as np
import pandas as pd

import tensorflow as tf
from collections import deque


In [2]:
from utils import *


In [3]:
sys.argv = [
    'inference.py',                
    '--cam',          'webcam',
    '--flipcam','False',
    '--dataset',      'cifar10',
    '--model_name',   'custom',
    '--train_model',  'True',         
    '--epochs',       '50',               
    '--lr',           '0.005',                 
    '--batch_size',   '64',            
    '--path_models',  'models',    
    '--path_results', 'results',    
    '--verbose',      'True',
]

In [4]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--train_model',  type=bool,  default=True,     help='Set to True to train the model.')
parser.add_argument('--cam',          type=str,   default='webcam', help='Camera to be used.')
parser.add_argument('--flipcam',          type=bool,   default=False, help='Camera to be used.')
parser.add_argument('--dataset',      type=str,   default='cifar10',help='Camera to be used.')
parser.add_argument('--model_name',   type=str,   default='custom', help='Model to be used.')
parser.add_argument('--epochs',       type=int,   default=50,       help='Number of epochs for training.')
parser.add_argument('--lr',           type=float, default=0.005,    help='Learning rate for the optimizer.')
parser.add_argument('--batch_size',   type=int,   default=32,       help='Batch size for training.')
parser.add_argument('--path_models',  type=str,   required=True,    help='Path to tarined models.')
parser.add_argument('--path_results', type=str,   required=True,    help='Path to save/saved results.')
parser.add_argument('--verbose',      type=bool,  required=True,    help='show progress.')
args = parser.parse_args()


In [5]:
dataset_manager = DatasetManager(dataset_name='cifar10',preprocess_data=True)
class_names = dataset_manager.get_class_names()

## Load Model

In [6]:
model_name = f'{args.path_models}/Resnet_pretrained_{args.epochs}_{args.lr}.keras'
if os.path.exists(model_name):
    model = tf.keras.models.load_model(model_name)
    history_df = pd.read_csv(f'{args.path_models}/training_history_{args.epochs}_{args.lr}.csv')
    print('-'*120)
    print(f'model loaded for :{args.epochs} epochs & learn_rate: {args.lr}')
    print('-'*120)
else:
    print(f'model not found :{args.epochs} epochs & learn_rate: {args.lr}')

------------------------------------------------------------------------------------------------------------------------
model loaded for :50 epochs & learn_rate: 0.005
------------------------------------------------------------------------------------------------------------------------


## Model Incpection

In [7]:
#### PRE TRAINING INSPECTION
inspector= Inspection(model)
inspector.model_stats(print_summary=False)

Trainable Parameters           550570
Non-Trainable Parameters       0.0
Total Parameters:              550570.0
Model Size (MB)                2.10
Approximate FLOPs              38.90 MFLOPs


## Getting Ready for Inference

In [8]:
infer = Inference(dataset_name='cifar10', model=model)

## Transfer Converted Model and Inference Code to Raspberry Pi

## Select Camera Type

In [11]:
# Use the IP and port displayed in EpocCam or NDI app
# iphone_url = "http://192.168.1.4:4747/video"
args.cam =  'local' # webcam phone, raspi

cam_location = 0 if args.cam =='local' else "http://172.20.10.5:4747/video"   #http://192.168.1.4:4747/video
print(f'CAM Type: {args.cam:<15} CAM Location: {cam_location:<20}')


CAM Type: local           CAM Location: 0                   


## Test Camera

In [12]:
test_camera       = True
print_camera_info = False

args.flipcam = False
if test_camera:
    cap = cv2.VideoCapture(cam_location)
    while True:
        ret, frame = cap.read()
        if args.flipcam:
            frame = cv2.flip(frame, 1)
        if not ret:
            print('camera failed or not available')
            break
        cv2.imshow("Webcam Prediction with CIFAR-10 Class Box", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [13]:
from dataclasses import dataclass
@dataclass
class Parameters:
    box_width      : int   = 180
    box_height     : int   = 20 * len(class_names) + 40
    color_green    : tuple = (0, 255, 0)
    color_white    : tuple = (255, 255, 255)
    color_black    : tuple = (0, 0, 0)
    fps   : int = 0

params = Parameters()

In [ ]:
# # Open the video stream
# cap = cv2.VideoCapture(iphone_url)

cap = cv2.VideoCapture(cam_location)                           # 0 for the primary webcam

prev_time = time.time()
confidence_history = {label: deque(maxlen=10) for label in class_names}

while True:
    ret, frame = cap.read()  
    if not ret:
        print("Failed to retrieve frame")
        break
    
    # Predict on the captured frame
    predictions = infer.predict_frame(frame)
    if args.flipcam:
        frame = cv2.flip(frame, 1)
    # Update confidence history for each class with smoothing
    smooth_confidences = []
    for i, label in enumerate(class_names):
        confidence_history[label].append(predictions[i])
        smooth_conf = np.mean(confidence_history[label])
        smooth_confidences.append(smooth_conf)

    # Get the top predicted class and its smoothed confidence
    class_id = np.argmax(smooth_confidences)
    class_name = class_names[class_id]
    smooth_confidence = smooth_confidences[class_id]

    # Calculate FPS
    current_time = time.time()
    fps = 1 / (current_time - prev_time)
    prev_time = current_time

    # Draw semi-transparent box on the left
    overlay = frame.copy()
    
    cv2.rectangle(overlay, (0, 0), (params.box_width, params.box_height), params.color_black, -1)
    frame = cv2.addWeighted(overlay, 0.4, frame, 0.6, 0)  # Add transparency

    # Display class labels and prediction confidences in the box
    for i, (label, smooth_conf) in enumerate(zip(class_names, smooth_confidences)):
        text = f"{label}: {smooth_conf:.2f}"
        y_position = 20 * (i + 1) + 10
        color = params.color_green if i == class_id else params.color_white  # Highlight top class
        cv2.putText(frame, text, (10, y_position), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Display the main class prediction, confidence, and FPS on the frame
#     text = f"Top Class: {class_name}, Confidence: {smooth_confidence:.2f}, FPS: {fps:.2f}"
#     cv2.putText(frame, text, (10, box_height + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    # Show the frame with the prediction, smoothed confidence, and FPS
    cv2.imshow("Webcam Prediction with CIFAR-10 Class Box", frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

## INFERENCE USING PHONE CAMERA

In [35]:
# while True:
#     ret, frame = cap.read()
#     if not ret:
        
#         break

#     # Display the frame
#     cv2.imshow('iPhone Camera Stream', frame)

#     # Press 'q' to quit
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()
